# FOOOF - Model Description

In FOOOF, an electrophysiological PSD is fit as a combination of the background process (the arythmic, background 1/f), upon which there may be oscillations - frequency regions in which there are 'bumps', with power over and above this background process.

This formulation roughly translates to fitting the PSD as:

$$psd = BG + \sum_{n=0}^{N} G_n$$

Where $BG$ is the 'background fit' and each $G_n$ is a gaussian fit to a putative oscillation, for $N$ total oscillations extracted from the PSD.

#### Background

The background fit is a log-lorentzian, fit on the semilog PSD (linear frequencies and log10 power values). 

The log-lorentzian is of the form:

$$BG = 10^a * \frac{1}{(b + f^\chi)}$$

Or, equivalently:

$$BG = a - \log(b + f^\chi)$$

In this formulation, the 3 parameters $a$, $b$, and $\chi$ define the background, as:
- $a$ is the 'offset'
- $b$ relates to the 'knee'
- $\chi$ is the 'slope'

#### Oscillations

Regions of power over above this background activity, as defined above, are considered to be putative oscillations, and fit in the model by a gaussian. 

For each gaussian, $n$, with the form:

$$G_n = amp * exp\Bigl(\frac{- (x - ctr)^2}{2 * wdt^2}\Bigl)$$

The oscillation is defined in terms of the $amp$, $ctr$ and $wdt$, where:
- $amp$ is the amplitude of the oscillation, over and above the background
- $ctr$ is the center frequency of the oscillation
- $wdt$ is the width of the center frequency

The full PSD fit is therefore the combination of the background fit, and $N$ 'oscillatory regions', where each $n$ is a gaussian process found gaussian processes.

Full method details available in the paper: [LINK WHEN AVAILABLE]

## FOOOF Settings

With default settings, FOOOF is minimally constrained. It defaults as such since there are not universal settings that work across all different dataset modalities. Appropriate settings also vary with PSD quality (noise, or effectively, the smoothness of the PSD), and frequency ranges. 

For any given dataset, FOOOF will likely need some tuning of parameters for optimal performance. 

To do so, we suggest using a combination of the following considerations:
- A priori constraints, given your data, such as the number of oscillations you expect to extract
- Qualitative analysis, guided by examing the the plotted model fit results, as compared to input data
- Quantitative analysis, considering the model fit and error (however, see note at the bottom regarding interpreting model fit error)

Tuning FOOOF is an imperfect art, and should be done carefully, as assumptions built into the settings chosen will impact the model results. For this reason, FOOOF settings should not be changed between PSDs (across channels, trials, or subjects), if they are to be meaningfully compared. We recommend 

### Options

At initialization, you have the following settings available:

#### bandwidth_limits (Hz)

Enforced limits on the bandwidths of extracted oscillations, given as a list of [minimum bandwidth, maximum bandwidth]. We recommend bandwidths at last twice the frequency resolution of the power spectrum.

Default: [0.5, 12]

#### max_n_oscs (int)

The maximum number of oscillations that can be extracted from a given PSD. FOOOF will halt searching for new oscillations when this number is reached. Note that FOOOF extracts oscillations iteratively by amplitude (over and above the background), and so this approach should extract the _n_ largest oscillations. 

Default: infinite

#### amp_std_thresh (in units of standard deviation)

The threshold, in terms of standard deviation of the background removed power spectrum, above which a data point must pass to be considered a candidate oscillation. 

Default: 2.0

#### min_amp (in units of the psd)

The minimum amplitude, above the background fit, that an oscillation must have to be extracted in the initial fit stage. This setting can be useful for datasets in which their may be flat PSDs, since in this case the standard deviation may be very low, leading to overfitting of small bumps as oscillations. Note that because this constraint is enforced during oscillation search, returned oscillations are not guaranteed to surpass this value in amplitude.

Default: 0

#### bg_use_knee (boolean)

Whether to fit a 'knee' parameter in the log-lorentzian background fit. 

Default: False

### A note on amplitudes

Note that there are two different parameters controlling FOOOF halting its search for oscillations based on amplitude. By default, there is no minimum absolute amplitude, but FOOOF will halt when new potential oscillations fail to pass the threshold, defined in units of standard deviation over and above the background. If both are defined, FOOOF will halt finding new oscillations when candidate fails to pass either the minimum amplitude, or the threshold. 


### Updating Settings

If you wish to change these settings, re-initialize the FOOOF object with new settings. Simply resetting the relevant attribute may not appropriately propragate the value, and may fail (either by erroring out, or not applying the settings properly during fit and returning erroneous results). 